In [1]:
# Library Imports.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Allows plots to appear directly in the notebook.
%matplotlib inline

from patsy import dmatrices
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score 

import pickle

In [2]:
# Read CSV files into Data Frames:
weather = pd.read_csv('working_weatherHistoryResampled_asof_2022.04.06.csv', keep_default_na=True, delimiter=',', skipinitialspace=True)
station = pd.read_csv('working_stationStateResampled_asof_2022.04.06.csv', keep_default_na=True, delimiter=',', skipinitialspace=True)

In [3]:
weather.shape

(684, 7)

In [4]:
station.shape

(75240, 6)

In [5]:
weather.head()

,weatherHour,latitude,longitude,description,temp,humidity,wind_speed
0,2022-02-22 12:00:00,53.35,-6.26,few clouds,282.81,71,10.80
1,2022-02-24 10:00:00,53.35,-6.26,scattered clouds,276.31,80,9.77
2,2022-03-01 09:00:00,53.35,-6.26,clear sky,277.21,86,0.98
3,2022-03-01 10:00:00,53.35,-6.26,scattered clouds,279.13,83,1.78
4,2022-03-01 11:00:00,53.35,-6.26,broken clouds,280.61,80,3.65


In [6]:
station.head()

,ID,stationId,weatherHour,status,available_bike_stands,available_bikes
0,143661,1,2022-02-22 12:00:00,OPEN,12,18
1,143662,2,2022-02-22 12:00:00,OPEN,18,2
2,143663,3,2022-02-22 12:00:00,OPEN,20,13
3,143664,4,2022-02-22 12:00:00,OPEN,22,13
4,143665,5,2022-02-22 12:00:00,OPEN,38,2


In [7]:
stationWeather = pd.merge(station, weather, how="left", on=["weatherHour"])

In [8]:
stationWeather.head()

,ID,stationId,weatherHour,status,available_bike_stands,available_bikes,latitude,longitude,description,temp,humidity,wind_speed
0,143661,1,2022-02-22 12:00:00,OPEN,12,18,53.35,-6.26,few clouds,282.81,71,10.8
1,143662,2,2022-02-22 12:00:00,OPEN,18,2,53.35,-6.26,few clouds,282.81,71,10.8
2,143663,3,2022-02-22 12:00:00,OPEN,20,13,53.35,-6.26,few clouds,282.81,71,10.8
3,143664,4,2022-02-22 12:00:00,OPEN,22,13,53.35,-6.26,few clouds,282.81,71,10.8
4,143665,5,2022-02-22 12:00:00,OPEN,38,2,53.35,-6.26,few clouds,282.81,71,10.8


In [9]:
stationWeather.tail()

,ID,stationId,weatherHour,status,available_bike_stands,available_bikes,latitude,longitude,description,temp,humidity,wind_speed
75235,218896,106,2022-04-04 10:00:00,OPEN,3,17,53.35,-6.26,broken clouds,285.57,86,10.84
75236,218897,107,2022-04-04 10:00:00,OPEN,29,11,53.35,-6.26,broken clouds,285.57,86,10.84
75237,218898,108,2022-04-04 10:00:00,OPEN,37,3,53.35,-6.26,broken clouds,285.57,86,10.84
75238,218899,109,2022-04-04 10:00:00,OPEN,4,24,53.35,-6.26,broken clouds,285.57,86,10.84
75239,218900,110,2022-04-04 10:00:00,OPEN,28,2,53.35,-6.26,broken clouds,285.57,86,10.84


In [10]:
stationWeather.dtypes

ID                         int64
stationId                  int64
weatherHour               object
status                    object
available_bike_stands      int64
available_bikes            int64
latitude                 float64
longitude                float64
description               object
temp                     float64
humidity                   int64
wind_speed               float64
dtype: object

In [11]:
stationWeather[['weatherDate', 'weatherHour']] = stationWeather['weatherHour'].str.split(' ', expand=True)
stationWeather

,ID,stationId,weatherHour,status,available_bike_stands,available_bikes,latitude,longitude,description,temp,humidity,wind_speed,weatherDate
0,143661,1,12:00:00,OPEN,12,18,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22
1,143662,2,12:00:00,OPEN,18,2,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22
2,143663,3,12:00:00,OPEN,20,13,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22
3,143664,4,12:00:00,OPEN,22,13,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22
4,143665,5,12:00:00,OPEN,38,2,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75235,218896,106,10:00:00,OPEN,3,17,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04
75236,218897,107,10:00:00,OPEN,29,11,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04
75237,218898,108,10:00:00,OPEN,37,3,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04
75238,218899,109,10:00:00,OPEN,4,24,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04


In [12]:
stationWeather[['weatherHour', 'weatherMinute', 'weatherSecond']] = stationWeather['weatherHour'].str.split(':', expand=True)
stationWeather

,ID,stationId,weatherHour,status,available_bike_stands,available_bikes,latitude,longitude,description,temp,humidity,wind_speed,weatherDate,weatherMinute,weatherSecond
0,143661,1,12,OPEN,12,18,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00
1,143662,2,12,OPEN,18,2,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00
2,143663,3,12,OPEN,20,13,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00
3,143664,4,12,OPEN,22,13,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00
4,143665,5,12,OPEN,38,2,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75235,218896,106,10,OPEN,3,17,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00
75236,218897,107,10,OPEN,29,11,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00
75237,218898,108,10,OPEN,37,3,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00
75238,218899,109,10,OPEN,4,24,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00


In [13]:
stationWeather['cal_bike_stands'] = stationWeather.available_bike_stands + stationWeather.available_bikes
stationWeather

,ID,stationId,weatherHour,status,available_bike_stands,available_bikes,latitude,longitude,description,temp,humidity,wind_speed,weatherDate,weatherMinute,weatherSecond,cal_bike_stands
0,143661,1,12,OPEN,12,18,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00,30
1,143662,2,12,OPEN,18,2,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00,20
2,143663,3,12,OPEN,20,13,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00,33
3,143664,4,12,OPEN,22,13,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00,35
4,143665,5,12,OPEN,38,2,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75235,218896,106,10,OPEN,3,17,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00,20
75236,218897,107,10,OPEN,29,11,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00,40
75237,218898,108,10,OPEN,37,3,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00,40
75238,218899,109,10,OPEN,4,24,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00,28


In [14]:
# Change description to category type for numeric encoding
stationWeather['description'] = stationWeather['description'].astype('category')
stationWeather.dtypes

ID                          int64
stationId                   int64
weatherHour                object
status                     object
available_bike_stands       int64
available_bikes             int64
latitude                  float64
longitude                 float64
description              category
temp                      float64
humidity                    int64
wind_speed                float64
weatherDate                object
weatherMinute              object
weatherSecond              object
cal_bike_stands             int64
dtype: object

In [15]:
# Catergorical Encoding
stationWeather['num_desc'] = stationWeather['description'].cat.codes
stationWeather

,ID,stationId,weatherHour,status,available_bike_stands,available_bikes,latitude,longitude,description,temp,humidity,wind_speed,weatherDate,weatherMinute,weatherSecond,cal_bike_stands,num_desc
0,143661,1,12,OPEN,12,18,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00,30,2
1,143662,2,12,OPEN,18,2,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00,20,2
2,143663,3,12,OPEN,20,13,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00,33,2
3,143664,4,12,OPEN,22,13,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00,35,2
4,143665,5,12,OPEN,38,2,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00,40,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75235,218896,106,10,OPEN,3,17,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00,20,0
75236,218897,107,10,OPEN,29,11,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00,40,0
75237,218898,108,10,OPEN,37,3,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00,40,0
75238,218899,109,10,OPEN,4,24,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00,28,0


In [16]:
station1Weather = stationWeather.query('stationId == 1')
station1Weather

,ID,stationId,weatherHour,status,available_bike_stands,available_bikes,latitude,longitude,description,temp,humidity,wind_speed,weatherDate,weatherMinute,weatherSecond,cal_bike_stands,num_desc
0,143661,1,12,OPEN,12,18,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00,30,2
110,143771,1,10,OPEN,17,13,53.35,-6.26,scattered clouds,276.31,80,9.77,2022-02-24,00,00,30,13
220,143881,1,09,OPEN,8,22,53.35,-6.26,clear sky,277.21,86,0.98,2022-03-01,00,00,30,1
330,143991,1,10,OPEN,12,18,53.35,-6.26,scattered clouds,279.13,83,1.78,2022-03-01,00,00,30,13
440,144101,1,11,OPEN,14,16,53.35,-6.26,broken clouds,280.61,80,3.65,2022-03-01,00,00,30,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74690,218351,1,06,OPEN,17,13,53.35,-6.26,broken clouds,283.60,91,9.39,2022-04-04,00,00,30,0
74800,218461,1,07,OPEN,22,8,53.35,-6.26,broken clouds,283.86,90,8.45,2022-04-04,00,00,30,0
74910,218571,1,08,OPEN,24,5,53.35,-6.26,broken clouds,284.18,89,9.12,2022-04-04,00,00,29,0
75020,218681,1,09,OPEN,26,3,53.35,-6.26,broken clouds,284.66,89,10.12,2022-04-04,00,00,29,0


In [17]:
# Checking correlations for all the contious features
station1Weather.corr()

,ID,stationId,available_bike_stands,available_bikes,latitude,longitude,temp,humidity,wind_speed,cal_bike_stands,num_desc
ID,1.000000,NaN,0.171404,-0.171881,NaN,NaN,0.230413,-0.141198,-0.238943,-0.006992,0.008602
stationId,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
available_bike_stands,0.171404,NaN,1.000000,-0.998649,NaN,NaN,0.415894,-0.290735,0.214522,0.038776,-0.061299
available_bikes,-0.171881,NaN,-0.998649,1.000000,NaN,NaN,-0.414172,0.288218,-0.215903,0.013194,0.059812
latitude,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
longitude,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
temp,0.230413,NaN,0.415894,-0.414172,NaN,NaN,1.000000,-0.467591,0.136850,0.038439,-0.033805
humidity,-0.141198,NaN,-0.290735,0.288218,NaN,NaN,-0.467591,1.000000,-0.178957,-0.052140,-0.049107
wind_speed,-0.238943,NaN,0.214522,-0.215903,NaN,NaN,0.136850,-0.178957,1.000000,-0.023807,0.011951
cal_bike_stands,-0.006992,NaN,0.038776,0.013194,NaN,NaN,0.038439,-0.052140,-0.023807,1.000000,-0.029395


# Random Forest

In [18]:
X = station1Weather[['stationId', 'weatherHour', 'temp', 'humidity', 'wind_speed', 'cal_bike_stands', 'num_desc']]
y = station1Weather.available_bikes

<h2>Training</h2>

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [20]:
random_forest = RandomForestRegressor(n_estimators = 1000, random_state = 42)
random_forest.fit(X_train, y_train)

RandomForestRegressor(n_estimators=1000, random_state=42)

In [22]:
# Serialize model object into a file called model.pkl on disk using pickle
with open('dwmb_resample_singleStation_randomForest_model.pkl', 'wb') as handle:
    pickle.dump(random_forest, handle, pickle.HIGHEST_PROTOCOL)

<h2>Testing</h2>

In [24]:
randomForest_predictions = random_forest.predict(X_test)
print(type(randomForest_predictions))


<class 'numpy.ndarray'>


In [26]:
df = pd.DataFrame({'Actual':y_test, 'Predicted':randomForest_predictions})
df

,Actual,Predicted
12430,27,26.978
41580,24,24.245
33330,13,13.065
55440,14,10.784
33110,20,19.056
...,...,...
71170,25,20.233
55330,19,19.700
54780,16,21.298
32230,22,23.335


In [29]:
# Metrics for the Model
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, randomForest_predictions))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, randomForest_predictions))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, randomForest_predictions)))

Mean Absolute Error: 4.3866553398058254
Mean Squared Error: 31.172214791262128
Root Mean Squared Error: 5.583208288364507


In [30]:
# Calculate the absolute errors
errors = abs(randomForest_predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Mean Absolute Error: 4.39 degrees.
Accuracy: 53.86 %.


In [32]:
# Function to output evaluation metrics
def printMetrics(testActualVal, predictions):
    #classification evaluation measures
    print('\n==============================================================================')
    print("MAE: ", metrics.mean_absolute_error(testActualVal, predictions))
    #print("MSE: ", metrics.mean_squared_error(testActualVal, predictions))
    print("RMSE: ", metrics.mean_squared_error(testActualVal, predictions)**0.5)
    print("R2: ", metrics.r2_score(testActualVal, predictions))

In [35]:
printMetrics(y_test, randomForest_predictions)


MAE:  4.3866553398058254
RMSE:  5.583208288364507
R2:  0.4752455499499375
